In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12718689167979090650,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7334306784
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10270585445815653399
 physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"]

# Module

In [1]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

### Dataset

In [2]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [3]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [4]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [5]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [7]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [9]:
%%time 

model.fit(x_train_reshaped, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 105us/sample - loss: 0.1954 - accuracy: 0.9406
Epoch 2/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0644 - accuracy: 0.9800
Epoch 3/10
60000/60000 [==============================] - 3s 50us/sample - loss: 0.0469 - accuracy: 0.9848
Epoch 4/10
60000/60000 [==============================] - 3s 47us/sample - loss: 0.0375 - accuracy: 0.9879
Epoch 5/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0309 - accuracy: 0.9906
Epoch 6/10
60000/60000 [==============================] - 3s 58us/sample - loss: 0.0251 - accuracy: 0.9919
Epoch 7/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0210 - accuracy: 0.9935
Epoch 8/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0177 - accuracy: 0.9943
Epoch 9/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0151 - accuracy: 0.9953
Epoch 10/10
6

# Model Evaluation

In [10]:
predictions = model.predict(x_test_reshaped)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy*100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on benign test example: 99.07000000000001%


# Adversarial Examples

In [11]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=True)

## 1-1) JSMA_origin
### 원본 이미지 x_test

In [28]:
y_test_mix = y_test.copy()
np.random.shuffle(y_test_mix)
y_test_mix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
y_test_mix = tf.keras.utils.to_categorical(y_test_mix, 10)

In [21]:
y_test_norm = tf.keras.utils.to_categorical(y_test, 10)

In [29]:
y_test_mix.shape

(10000, 10)

In [30]:
y_test_norm.shape

(10000, 10, 10)

In [31]:
y_test.shape

(10000, 10)

In [15]:
%%time
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_reshaped)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 164.14149451255798


In [33]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.180000215768814%


## 1-2) JSMA_noise
### noise image -> x_test_noise

In [18]:
# noise 
noise = np.random.normal(0,0.01,x_test.shape)
x_test_noise1 = x_test_norm + noise
x_test_noise1_reshape = x_test_noise1.reshape(-1,28,28,1)

In [19]:
%%time
from art.attacks.evasion import SaliencyMapMethod

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise1_reshape)

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

CPU times: user 2min 47s, sys: 9.23 s, total: 2min 56s
Wall time: 2min 14s


In [34]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.180000215768814%


## noise 2 - sigma-0.03

In [20]:
# noise 2
noise2 = np.random.normal(0,0.03,x_test.shape)
x_test_noise2 = x_test_norm + noise2
x_test_noise2_reshape = x_test_noise2.reshape(-1,28,28,1)

In [21]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise2_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 93.67624640464783


In [22]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 9.430000185966492%


## noise 3 - sigma-0.05

In [23]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [24]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise3_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 122.24460339546204


In [25]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.260000079870224%


## noise 4 - sigma-0.07

In [26]:
# noise 4
noise4 = np.random.normal(0,0.07,x_test.shape)
x_test_noise4 = x_test_norm + noise4
x_test_noise4_reshape = x_test_noise4.reshape(-1,28,28,1)

In [27]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise4_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 117.07772278785706


In [28]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 9.950000047683716%


## noise 5 - sigma-0.1

In [29]:
# noise 5
noise5 = np.random.normal(0,0.1,x_test.shape)
x_test_noise5 = x_test_norm + noise5
x_test_noise5_reshape = x_test_noise5.reshape(-1,28,28,1)

In [30]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise5_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 115.22520875930786


In [31]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 9.640000015497208%


## noise 6 - sigma-0.2

In [32]:
# noise 6
noise6 = np.random.normal(0,0.2,x_test.shape)
x_test_noise6 = x_test_norm + noise6
x_test_noise6_reshape = x_test_noise6.reshape(-1,28,28,1)

In [33]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise6_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 90.65211510658264


In [38]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 9.98999997973442%


# 1-2. JSMA theta=0.1, gamma=0.5

## 1-1) JSMA_origin
### 원본 이미지 x_test

In [34]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_reshaped)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 218.4063220024109


In [36]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 9.470000118017197%


## 1-2) JSMA_noise
### noise image -> x_test_noise

In [37]:
# noise 
noise = np.random.normal(0,0.01,x_test.shape)
x_test_noise1 = x_test_norm + noise
x_test_noise1_reshape = x_test_noise1.reshape(-1,28,28,1)

In [38]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise1_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 261.09409046173096


In [39]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.130000114440918%


## noise 2 - sigma-0.03

In [40]:
# noise 2
noise2 = np.random.normal(0,0.03,x_test.shape)
x_test_noise2 = x_test_norm + noise2
x_test_noise2_reshape = x_test_noise2.reshape(-1,28,28,1)

In [41]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise2_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 292.5268840789795


In [42]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.39000004529953%


## noise 3 - sigma-0.05

In [43]:
# noise 3
noise3 = np.random.normal(0,0.05,x_test.shape)
x_test_noise3 = x_test_norm + noise3
x_test_noise3_reshape = x_test_noise3.reshape(-1,28,28,1)

In [44]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise3_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 291.8446452617645


In [45]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.559999942779541%


## noise 4 - sigma-0.07

In [46]:
# noise 4
noise4 = np.random.normal(0,0.07,x_test.shape)
x_test_noise4 = x_test_norm + noise4
x_test_noise4_reshape = x_test_noise4.reshape(-1,28,28,1)

In [47]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise4_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 239.9981026649475


In [48]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.000000149011612%


## noise 5 - sigma-0.1

In [49]:
# noise 5
noise5 = np.random.normal(0,0.1,x_test.shape)
x_test_noise5 = x_test_norm + noise5
x_test_noise5_reshape = x_test_noise5.reshape(-1,28,28,1)

In [50]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise5_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 240.66079211235046


In [51]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.339999943971634%


## noise 6 - sigma-0.2

In [52]:
# noise 6
noise6 = np.random.normal(0,0.2,x_test.shape)
x_test_noise6 = x_test_norm + noise6
x_test_noise6_reshape = x_test_noise6.reshape(-1,28,28,1)

In [53]:
from art.attacks.evasion import SaliencyMapMethod

start = time.time() # 시작시간

#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator, theta=0.1,
                                     gamma=0.5,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_noise6_reshape)

print("time:",time.time()-start) # 실행시간

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

time: 191.36079239845276


In [54]:
loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.23000031709671%
